### import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.3.0


### Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir) # 해당 dir에 학습해둔 모델 있는지 확인
    if ckpt:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0
    
def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### Data load & pre-processing function

In [3]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N,28,28] -> [N,28,28,1]
    test_data = np.expand_dims(test_data, axis=-1) # [N,28,28] -> [N,28,28,1]
    
    train_data, test_data = normalize(train_data, test_data)
    
    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]
    
    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

### Performance function

In [4]:
def loss_func(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits,
                                                                   y_true=labels,
                                                                   from_logits=True)) # softmax를 거치지 않을때
    return loss

def accuracy_func(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_func(model, images, labels)
    return tape.gradient(loss, model.variables)

### Model function

In [5]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init):
    return tf.keras.layers.Dense(units=label_dim, # 출력값의 크기
                                 use_bias=True, # 편향 사용 유무
                                 kernel_initializer=weight_init) # 가중치 초기화 함수

def sigmoid():
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

### Create model(class version)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
#         weight_init = tf.keras.initializers.RandomNormal()
        weight_init = tf.keras.initializers.glorot_uniform() 
        # Sigmoid : Xavier 초기화
        # Relu : He 초기화
        
        self.model = tf.keras.Sequential()
        self.model.add(flatten())
        
        for i in range(2):
            self.model.add(dense(512, weight_init))
#             self.model.add(relu())
            self.model.add(sigmoid())
            
        self.model.add(dense(label_dim, weight_init))
        
    def call(self, x, training=None, mask=None):
        x = self.model(x)
        return x

### Create model(function version)

In [7]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.glorot_uniform()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(512, weight_init))
        model.add(relu())
#         model.add(sigmoid())
        
    model.add(dense(label_dim, weight_init))
    
    return model

### Define data & hyper-parameter

In [8]:
# dataset
train_x, train_y, test_x, test_y = load_mnist()

# parameters
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10
train_flag = True

# Graph input using Dataset API
# prefetch : data 로드 시간을 줄이기 위해 미리 batch size 만큼 적재시킴
# drop_remainder : 마지막 남은 data를 drop 여부
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
                                                   shuffle(buffer_size=100000).\
                                                   prefetch(buffer_size=batch_size).\
                                                   batch(batch_size, drop_remainder=True) 


test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
                                                  shuffle(buffer_size=100000).\
                                                  prefetch(buffer_size=len(test_x)).\
                                                  batch(len(test_x))

### Define model & optimizer & writer

In [9]:
# Model
network = create_model_function(label_dim)

# Training
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Writer
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_deep'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [10]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
        
    # train phase
    with summary_writer.as_default():
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))
                
                train_loss = loss_func(network, train_input, train_label)
                train_accuracy = accuracy_func(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_func(network, test_input, test_label)
                    
                tf.summary.scalar(name='train_lass', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
                
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))   
# test phase
else:
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:
        test_accuracy = accuracy_func(network, test_input, test_label)
        
    print("test Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.5280, train_loss: 1.91824627, train_accuracy: 0.6328, test_Accuracy: 0.3975
Epoch: [ 0] [    1/  468] time: 0.7095, train_loss: 1.76402164, train_accuracy: 0.7344, test_Accuracy: 0.5901
Epoch: [ 0] [    2/  468] time: 0.8998, train_loss: 1.52774596, train_accuracy: 0.7656, test_Accuracy: 0.6590
Epoch: [ 0] [    3/  468] time: 1.1575, train_loss: 1.32866633, train_accuracy: 0.8047, test_Accuracy: 0.7036
Epoch: [ 0] [    4/  468] time: 1.3791, train_loss: 1.17321265, train_accuracy: 0.7891, test_Accuracy: 0.7340
Epoch: [ 0] [    5/  468] time: 1.5626, train_loss: 1.05256009, train_accuracy: 0.7734, test_Accuracy: 0.7468
Epoch: [ 0] [    6/  468] time: 1.7558, train_loss: 0.85539317, train_accuracy: 0.8203, test_Accuracy: 0.7681
Epoch: [ 0] [    7/  468] time: 1.9286, train_loss: 0.90408528, train_accuracy: 0.7422, test_Accuracy: 0.7884
Epoch: [ 0] [    8/  468] time: 2.1169,

Epoch: [ 0] [   74/  468] time: 14.6976, train_loss: 0.28594425, train_accuracy: 0.9531, test_Accuracy: 0.9239
Epoch: [ 0] [   75/  468] time: 14.9142, train_loss: 0.21194312, train_accuracy: 0.9297, test_Accuracy: 0.9207
Epoch: [ 0] [   76/  468] time: 15.1329, train_loss: 0.30953267, train_accuracy: 0.9062, test_Accuracy: 0.9212
Epoch: [ 0] [   77/  468] time: 15.3651, train_loss: 0.26655871, train_accuracy: 0.9297, test_Accuracy: 0.9240
Epoch: [ 0] [   78/  468] time: 15.5984, train_loss: 0.26422709, train_accuracy: 0.9062, test_Accuracy: 0.9271
Epoch: [ 0] [   79/  468] time: 15.8141, train_loss: 0.26488167, train_accuracy: 0.9062, test_Accuracy: 0.9268
Epoch: [ 0] [   80/  468] time: 16.0200, train_loss: 0.26888210, train_accuracy: 0.9062, test_Accuracy: 0.9257
Epoch: [ 0] [   81/  468] time: 16.2202, train_loss: 0.28426728, train_accuracy: 0.9375, test_Accuracy: 0.9285
Epoch: [ 0] [   82/  468] time: 16.4398, train_loss: 0.31047243, train_accuracy: 0.9375, test_Accuracy: 0.9310
E

Epoch: [ 0] [  148/  468] time: 30.7420, train_loss: 0.17235419, train_accuracy: 0.9688, test_Accuracy: 0.9435
Epoch: [ 0] [  149/  468] time: 30.9577, train_loss: 0.12007131, train_accuracy: 0.9609, test_Accuracy: 0.9409
Epoch: [ 0] [  150/  468] time: 31.1842, train_loss: 0.14016688, train_accuracy: 0.9766, test_Accuracy: 0.9399
Epoch: [ 0] [  151/  468] time: 31.3989, train_loss: 0.20855805, train_accuracy: 0.9375, test_Accuracy: 0.9393
Epoch: [ 0] [  152/  468] time: 31.6429, train_loss: 0.24885365, train_accuracy: 0.9453, test_Accuracy: 0.9388
Epoch: [ 0] [  153/  468] time: 31.8664, train_loss: 0.09190559, train_accuracy: 0.9844, test_Accuracy: 0.9407
Epoch: [ 0] [  154/  468] time: 32.1231, train_loss: 0.17475387, train_accuracy: 0.9453, test_Accuracy: 0.9455
Epoch: [ 0] [  155/  468] time: 32.3602, train_loss: 0.10123867, train_accuracy: 0.9609, test_Accuracy: 0.9457
Epoch: [ 0] [  156/  468] time: 32.5877, train_loss: 0.17281108, train_accuracy: 0.9609, test_Accuracy: 0.9448
E

Epoch: [ 0] [  222/  468] time: 48.1139, train_loss: 0.06884724, train_accuracy: 0.9844, test_Accuracy: 0.9520
Epoch: [ 0] [  223/  468] time: 48.3266, train_loss: 0.12320329, train_accuracy: 0.9766, test_Accuracy: 0.9516
Epoch: [ 0] [  224/  468] time: 48.5424, train_loss: 0.07459537, train_accuracy: 0.9766, test_Accuracy: 0.9516
Epoch: [ 0] [  225/  468] time: 48.7444, train_loss: 0.12104429, train_accuracy: 0.9844, test_Accuracy: 0.9538
Epoch: [ 0] [  226/  468] time: 48.9445, train_loss: 0.12120435, train_accuracy: 0.9688, test_Accuracy: 0.9530
Epoch: [ 0] [  227/  468] time: 49.1611, train_loss: 0.16983669, train_accuracy: 0.9531, test_Accuracy: 0.9528
Epoch: [ 0] [  228/  468] time: 49.3836, train_loss: 0.07909270, train_accuracy: 0.9766, test_Accuracy: 0.9507
Epoch: [ 0] [  229/  468] time: 49.6042, train_loss: 0.09967741, train_accuracy: 0.9766, test_Accuracy: 0.9489
Epoch: [ 0] [  230/  468] time: 49.8180, train_loss: 0.18425542, train_accuracy: 0.9531, test_Accuracy: 0.9484
E

Epoch: [ 0] [  296/  468] time: 63.8158, train_loss: 0.08507487, train_accuracy: 0.9844, test_Accuracy: 0.9605
Epoch: [ 0] [  297/  468] time: 64.0412, train_loss: 0.14476874, train_accuracy: 0.9531, test_Accuracy: 0.9604
Epoch: [ 0] [  298/  468] time: 64.2687, train_loss: 0.11010981, train_accuracy: 0.9688, test_Accuracy: 0.9604
Epoch: [ 0] [  299/  468] time: 64.4863, train_loss: 0.08254175, train_accuracy: 0.9766, test_Accuracy: 0.9605
Epoch: [ 0] [  300/  468] time: 64.7049, train_loss: 0.12886882, train_accuracy: 0.9609, test_Accuracy: 0.9611
Epoch: [ 0] [  301/  468] time: 64.9264, train_loss: 0.19365260, train_accuracy: 0.9609, test_Accuracy: 0.9606
Epoch: [ 0] [  302/  468] time: 65.1294, train_loss: 0.09589194, train_accuracy: 0.9844, test_Accuracy: 0.9609
Epoch: [ 0] [  303/  468] time: 65.3403, train_loss: 0.09730618, train_accuracy: 0.9844, test_Accuracy: 0.9605
Epoch: [ 0] [  304/  468] time: 65.5403, train_loss: 0.17963110, train_accuracy: 0.9531, test_Accuracy: 0.9607
E

Epoch: [ 0] [  370/  468] time: 79.3049, train_loss: 0.09663107, train_accuracy: 0.9688, test_Accuracy: 0.9653
Epoch: [ 0] [  371/  468] time: 79.5284, train_loss: 0.10048800, train_accuracy: 0.9766, test_Accuracy: 0.9647
Epoch: [ 0] [  372/  468] time: 79.7529, train_loss: 0.10935427, train_accuracy: 0.9766, test_Accuracy: 0.9646
Epoch: [ 0] [  373/  468] time: 79.9656, train_loss: 0.06023980, train_accuracy: 0.9766, test_Accuracy: 0.9641
Epoch: [ 0] [  374/  468] time: 80.1735, train_loss: 0.11841727, train_accuracy: 0.9688, test_Accuracy: 0.9649
Epoch: [ 0] [  375/  468] time: 80.3912, train_loss: 0.07075916, train_accuracy: 0.9688, test_Accuracy: 0.9662
Epoch: [ 0] [  376/  468] time: 80.5991, train_loss: 0.06349541, train_accuracy: 0.9922, test_Accuracy: 0.9671
Epoch: [ 0] [  377/  468] time: 80.8177, train_loss: 0.17521951, train_accuracy: 0.9219, test_Accuracy: 0.9671
Epoch: [ 0] [  378/  468] time: 81.0392, train_loss: 0.07320998, train_accuracy: 0.9688, test_Accuracy: 0.9661
E

Epoch: [ 0] [  444/  468] time: 94.8370, train_loss: 0.12491389, train_accuracy: 0.9609, test_Accuracy: 0.9633
Epoch: [ 0] [  445/  468] time: 95.0400, train_loss: 0.05239653, train_accuracy: 0.9844, test_Accuracy: 0.9646
Epoch: [ 0] [  446/  468] time: 95.2352, train_loss: 0.08261348, train_accuracy: 0.9688, test_Accuracy: 0.9653
Epoch: [ 0] [  447/  468] time: 95.4391, train_loss: 0.12900200, train_accuracy: 0.9531, test_Accuracy: 0.9675
Epoch: [ 0] [  448/  468] time: 95.6412, train_loss: 0.11385743, train_accuracy: 0.9609, test_Accuracy: 0.9689
Epoch: [ 0] [  449/  468] time: 95.8412, train_loss: 0.10085145, train_accuracy: 0.9688, test_Accuracy: 0.9698
Epoch: [ 0] [  450/  468] time: 96.0423, train_loss: 0.07980422, train_accuracy: 0.9766, test_Accuracy: 0.9695
Epoch: [ 0] [  451/  468] time: 96.2541, train_loss: 0.20301923, train_accuracy: 0.9531, test_Accuracy: 0.9682
Epoch: [ 0] [  452/  468] time: 96.4503, train_loss: 0.13908377, train_accuracy: 0.9844, test_Accuracy: 0.9678
E

### Test accuracy : 96.64%